# 문제: 비행기 지연 예측

이 노트북의 여러 가지 목표는 다음과 같습니다.
- 다운로드한 ZIP 파일에서 데이터 집합 처리 및 생성
- 탐색적 데이터 분석(EDA)
- 기준 모형 마련
- 단순 모형에서 앙상블 모형으로 진행
- 하이퍼파라미터 최적화
- 특성 중요도 확인

## 비즈니스 시나리오 소개
여러분은 항공편이 지연된 승객의 고객 경험을 개선하기 위해 노력하는 여행 예약 웹 사이트에서 근무합니다. 이 회사는 고객이 미국에서 국내 여행객들로 가장 혼잡한 공항을 오가는 항공편을 예약할 때 날씨로 인해 항공편이 지연되는지 고객에게 알려줄 수 있는 기능을 만들려고 합니다. 

기계 학습을 활용하여 날씨로 인해 항공편이 지연될지 여부를 파악함으로써 이 문제의 일부를 해결해야 합니다. 여러분에게는 대형 항공사가 운영하는 국내선의 정시운항성 데이터 집합에 액세스할 수 있는 권한이 부여되었습니다. 이 데이터를 사용하여 가장 혼잡한 공항의 항공편이 지연될지 예측하는 기계 학습 모형을 훈련할 수 있습니다.

## 이 데이터 집합 소개
이 데이터 집합에는 국내 예약 승객 매출의 1% 이상을 차지하는 미국 공인 항공사에서 보고한 예정 및 실제 출발 및 도착 시간이 포함되어 있습니다. 이 데이터는 교통 통계국(BTS) 항공사 정보실에서 수집했습니다. 이 데이터 집합에는 2013년부터 2018년 사이 항공편의 날짜, 시간, 출발지, 목적지, 항공사, 거리 및 지연 상태가 포함되어 있습니다.

### 특성
이 데이터 집합의 특성에 대한 자세한 내용은 [On-time delay dataset features](https://www.transtats.bts.gov/Fields.asp)를 참조하십시오.

### 데이터 집합 속성  
웹 사이트: https://www.transtats.bts.gov/

본 실습에 사용된 데이터 집합은 미국 교통 통계국(BTS) 항공사 정보실에서 수집한 항공편 정시운항성 데이터를 컴파일한 것으로 https://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&amp;DB_URL=Mode_ID=1&amp;Mode_Desc=Aviation&amp;Subject_ID2=0에서 확인할 수 있습니다.

# 1단계: 문제 공식화 및 데이터 수집

이 시나리오의 비즈니스 문제와 달성하고자 하는 비즈니스 목표를 몇 문장으로 요약하여 아래에 작성하는 것으로 이 프로젝트를 시작하십시오. 팀에서 지향했으면 하는 비즈니스 지표를 포함합니다. 지표를 정의한 후 기계 학습 문제 기술서를 명확하게 작성합니다. 마지막으로, 이러한 문제에 해당하는 기계 학습의 유형에 대해 설명을 한 두 개 추가합니다. 

### <span style="color: blue;">프로젝트 프레젠테이션: 프로젝트 프레젠테이션에 이러한 세부 정보에 대한 요약을 포함합니다.</span>

### 1. 기계 학습이 배포에 적합한 솔루션인지와 그 이유 파악

In [ ]:
# 여기에 답변 작성

### 2. 비즈니스 문제, 성공 지표 및 원하는 기계 학습 결과 공식화

In [ ]:
# 여기에 답변 작성

### 3. 해결하려는 기계 학습 문제의 유형 식별

In [ ]:
# 여기에 답변 작성

### 4. 작업 중인 데이터의 적합성 분석

In [ ]:
# 여기에 답변 작성

### 설정

이제 어디에 에너지를 집중해야 할지 결정했으니, 문제 해결에 착수할 수 있도록 환경을 설정해 보겠습니다.

**참고:** 이 노트북은 `ml.m4.xlarge` 노트북 인스턴스에서 생성 및 검정되었습니다. 

In [ ]:
import os
from pathlib2 import Path
from zipfile import ZipFile
import time

import pandas as pd
import numpy as np
import subprocess

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# 2단계: 데이터 전처리 및 시각화  
이 데이터 전처리 단계에서는 데이터를 더 잘 이해할 수 있도록 데이터를 탐색하고 시각화할 수 있는 기회를 가져야 합니다. 먼저 필요한 라이브러리를 가져와서 데이터를 Pandas 데이터 프레임으로 읽어 들입니다. 그런 다음 데이터를 탐색합니다. 데이터 집합의 형태를 찾고 열과 작업 중인 열 유형(수치형, 범주형)을 탐색합니다. 특성에 대한 기본 통계를 수행하여 특성의 평균 및 범위를 파악하는 것이 좋습니다. 목표 열을 자세히 살펴보고 분포를 확인합니다.

### 고려해야 할 구체적인 질문
1. 특성에 대해 실행한 기본 통계에서 추론할 수 있는 것은 무엇인가요? 

2. 목표 클래스의 분포에서 추론할 수 있는 것은 무엇인가요?

3. 데이터를 탐색하면서 추론한 것이 또 있나요?

### <span style="color: blue;">프로젝트 프레젠테이션: 이러한 질문과 기타 유사한 질문에 대한 답변을 요약하여 프로젝트 프레젠테이션에 포함합니다.</span>

먼저 Amazon S3 퍼블릭 버킷에서 이 노트북 환경으로 데이터 집합을 가져옵니다.

In [ ]:
# 파일이 이미 원하는 경로에 있는지 아니면 다운로드해야 하는지 확인

base_path = '/home/ec2-user/SageMaker/project/data/FlightDelays/'
csv_base_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays/'
file_path = 'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2014_1.zip'

if not os.path.isfile(base_path + file_path):
    subprocess.run(['mkdir', '-p', base_path])
    subprocess.run(['mkdir', '-p', csv_base_path])
    subprocess.run(['aws', 's3', 'cp', 
                    's3://aws-tc-largeobjects/ILT-TF-200-MLDWTS/flight_delay_project/csvFlightData-5/', 
                    base_path,'--recursive'])
else:
    print('File already downloaded!')

In [ ]:
zip_files = [str(file) for file in list(Path(base_path).iterdir()) if '.zip' in str(file)]
len(zip_files)

#### ZIP 파일에서 CSV 파일 추출

In [ ]:
def zip2csv(zipFile_name , file_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays'):
    """
    zip 파일에서 csv 파일 추출
    zipFile_name: zip 파일 이름
    file_path: csv를 저장할 폴더 이름
    """
    try:
        with ZipFile(zipFile_name, 'r') as z: 
            print(f'Extracting {zipFile_name} ') 
            z.extractall(path=file_path) 
    except:
        print(f'zip2csv failed for {zipFile_name}')

for file in zip_files:
    zip2csv(file)

print("Files Extracted")

In [ ]:
csv_files = [str(file) for file in list(Path(csv_base_path).iterdir()) if '.csv' in str(file)]
len(csv_files)

CSV 파일을 로드하기 전에 추출된 폴더에서 HTML 파일을 읽습니다. 이 HTML 파일에는 데이터 집합에 포함된 특성에 대한 배경과 추가 정보가 포함되어 있습니다.

In [ ]:
from IPython.display import IFrame

IFrame(src="./data/csvFlightDelays/readme.html", width=1000, height=600)

### 샘플 CSV 다운로드

모든 CSV 파일을 결합하기 전에 CSV 파일 하나를 살펴보고 데이터를 파악합니다. Pandas를 사용하여 `On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_9.csv` 파일을 먼저 읽습니다. Python 기본 제공 함수인 `read_csv`를 사용할 수 있습니다([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)).

In [ ]:
df_temp = pd.read_csv(<CODE> # **여기에 코드 입력**

**질문**: 데이터 집합의 행 및 열 길이를 인쇄하고 열 이름을 인쇄합니다.

**힌트**: 데이터 프레임의 행과 열을 보려면 `<dataframe>.shape` 함수를 사용하고, 열 이름을 보려면`<dataframe>.columns`를 사용합니다.

In [ ]:
df_shape = # **여기에 코드 입력**
print(f'Rows and columns in one csv file is {df_shape}')

**질문**: 데이터 집합의 처음 10개 행을 인쇄합니다.  

**힌트**: Pandas 기본 제공 함수인 `head(x)`를 사용하여 `x`개의 행을 인쇄합니다.

In [ ]:
# 여기에 코드 입력

**질문**: 데이터 집합의 모든 열을 인쇄합니다. `<dataframe>.columns`를 사용하여 열 이름을 봅니다.

In [ ]:
print(f'The column names are :')
print('#########')
for col in <CODE>:# **여기에 코드 입력**
    print(col)

**질문**: 데이터 집합에서 'Del'이라는 단어가 포함된 열을 모두 인쇄합니다. 이렇게 하면 지연 데이터가 있는 열의 수를 확인할 수 있습니다.

**힌트**: Python list comprehension을 사용하여 특정 `if` 문 기준을 통과하는 값을 포함할 수 있습니다.

예: `[x for x in [1,2,3,4,5] if x > 2]`  

**힌트**: `in` 키워드([설명서](https://www.w3schools.com/python/ref_keyword_in.asp))를 사용하여 값이 목록에 있는지 확인할 수 있습니다. 

예: `5 in [1,2,3,4,5]`

In [1]:
# 여기에 코드 입력

다음은 데이터 집합에 대해 자세히 알아보는 데 도움이 되는 몇 가지 추가 질문입니다.

**질문*8   
1. 데이터 집합에는 행과 열이 몇 개 있나요?   
2. 데이터 집합에는 몇 년이 포함되어 있나요?   
3. 데이터 집합의 날짜 범위는 어떻게 되나요?   
4. 데이터 집합에는 어떤 항공사가 포함되었나요?   
5. 포함된 출발지 공항과 목적지 공항은 어디인가요?

In [ ]:
print("The #rows and #columns are ", <CODE> , " and ", <CODE>)
print("The years in this dataset are: ", <CODE>)
print("The months covered in this dataset are: ", <CODE>)
print("The date range for data is :" , min(<CODE>), " to ", max(<CODE>))
print("The airlines covered in this dataset are: ", list(<CODE>))
print("The Origin airports covered are: ", list(<CODE>))
print("The Destination airports covered are: ", list(<CODE>))

**질문**: 모든 출발지 및 도착지 공항의 수는 어떻게 되나요?

**힌트**: Pandas `values_count` 함수에서 `Origin` 및 `Dest` 열을 사용하여 각 공항의 값을 확인할 수 있습니다([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)).

In [ ]:
counts = pd.DataFrame({'Origin':<CODE>, 'Destination':<CODE>})
counts

**질문**: 데이터 집합의 항공편 수를 기준으로 상위 15개의 출발지 및 목적지 공항을 인쇄합니다.

**힌트**: Pandas `sort_values` 함수를 사용하면 됩니다([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)).

In [ ]:
counts.sort_values(by=<CODE>,ascending=False).head(15 )# 여기에 코드 입력

**질문**: 비행 여정에 대한 모든 정보를 종합해 볼 때 연착 여부를 예측할 수 있을까요?

In [ ]:
# 여기에 답변 입력

이제 샌프란시스코에서 로스앤젤레스로 출장을 간다고 가정해 보겠습니다. 로스앤젤레스에서의 예약을 더 잘 관리할 수 있도록 일련의 특성을 고려하여 항공편 지연에 대한 아이디어를 얻으려고 합니다. 이 데이터 집합에서 비행 전에 알 수 있는 특성은 몇 개인가요?

`DepDelay`, `ArrDelay`, `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay`, `LateAircraftDelay` 및 `DivArrDelay` 같은 열에는 지연에 대한 정보가 포함되어 있습니다. 하지만 이러한 지연은 출발지 또는 목적지에서 발생할 수 있습니다. 착륙 10분 전에 갑작스러운 기상 악화로 지연되는 경우 이 데이터는 로스앤젤레스의 예약을 관리하는 데 도움이 되지 않습니다.

따라서 문제 기술을 간소화하기 위해 다음 열을 고려하여 도착 지연을 예측하십시오.<br>

`Year`, `Quarter`, `Month`, `DayofMonth`, `DayOfWeek`, `FlightDate`, `Reporting_Airline`, `Origin`, `OriginState`, `Dest`, `DestState`, `CRSDepTime`, `DepDelayMinutes`, `DepartureDelayGroups`, `Cancelled`, `Diverted`, `Distance`, `DistanceGroup`, `ArrDelay`, `ArrDelayMinutes`, `ArrDel15`, `AirTime`

또한 출발지 및 목적지 공항을 다음과 같이 필터링합니다.
– 주요 공항: ATL, ORD, DFW, DEN, CLT, LAX, IAH, PHX, SFO
– 상위 5개 항공사: UA, OO, WN, AA, DL

이렇게 하면 결합할 CSV 파일 전체에서 데이터 크기를 줄이는 데 도움이 됩니다.

#### 모든 CSV 파일 결합

**힌트**:  
먼저 각 파일에서 개별 데이터 프레임을 복사하는 데 사용할 빈 데이터 프레임을 생성합니다. 그런 다음 `csv_files` 목록의 각 파일에 대해 다음을 수행합니다.

1. CSV 파일을 데이터 프레임으로 읽어 들입니다.  
2. `filter_cols` 변수를 기준으로 열을 필터링합니다.

```
        columns = ['col1', 'col2']
        df_filter = df[columns]
```

3. 각 subset_cols 내에 subset_vals만 유지합니다. `isin` Pandas 함수를 사용하여 `val`이 데이터 프레임 열에 있는지 확인한 다음 이를 포함하는 행을 선택합니다([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)).

```
        df_eg[df_eg['col1'].isin('5')]
```

4. 데이터 프레임과 빈 데이터 프레임을 연결합니다. 

In [ ]:
def combine_csv(csv_files, filter_cols, subset_cols, subset_vals, file_name = 'data/combined_files.csv'):
    """
    csv 파일을 하나의 데이터 프레임으로 결합합니다.
    csv_files: csv 파일 경로 목록
    filter_cols: 필터링할 열 목록
    subset_cols: 하위 집합 행에 대한 열 목록
    subset_vals: 하위 집합 행에 대한 값 목록
    """
    # 빈 데이터 프레임 생성
    df = # 여기에 코드 입력 
    
    for file in csv_files:
        # CSV 파일을 데이터 프레임에 읽어 들이기
        df_temp = pd.read_csv(<CODE>)# 여기에 코드 입력
        
        # filter_cols 변수를 기준으로 열 필터링
        # 예: columns = ['col1', 'col2']
        # df_filter = df[columns]
        df_temp = # 여기에 코드 입력
        
        # 각 subset_cols 내에 subset_vals만 유지
        #힌트: `isin` 함수를 사용하여 val이 데이터 프레임 열에 있는지 확인합니다.
        # 그런 다음 이를 포함하는 행을 선택합니다.
        # 예: df[df['col1'].isin('5')]
        for col, val in zip(subset_cols,subset_vals):
            df_temp = # 여기에 코드 입력     
        
        # Pandas concatenate `pd.concat`을 사용하여 주 데이터 프레임을 각 파일의 데이터 프레임과 연결합니다.
        df = pd.concat([df, df_temp], axis=0)
    
        
    df.to_csv(file_name, index=False)
    print(f'Combined csv stored at {file_name}')

In [ ]:
# cols는 도착 지연을 예측하기 위한 열 목록입니다. 
cols = ['Year','Quarter','Month','DayofMonth','DayOfWeek','FlightDate',
        'Reporting_Airline','Origin','OriginState','Dest','DestState',
        'CRSDepTime','Cancelled','Diverted','Distance','DistanceGroup',
        'ArrDelay','ArrDelayMinutes','ArrDel15','AirTime']

subset_cols = ['Origin', 'Dest', 'Reporting_Airline']

# subset_vals는 주요 출발지 및 목적지 공항과 상위 5개 항공사의 목록 모음입니다.
subset_vals = [['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'], 
               ['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'], 
               ['UA', 'OO', 'WN', 'AA', 'DL']]

위의 함수를 사용하여 다른 모든 파일을 쉽게 읽을 수 있는 단일 파일로 병합할 수 있습니다. 

**참고**: 완료하는 데 5~7분이 걸립니다.

In [ ]:
start = time.time()
combine_csv(csv_files, cols, subset_cols, subset_vals)
print(f'csv\'s merged in {round((time.time() - start)/60,2)} minutes')

#### 데이터 집합 로드

결합된 데이터 집합을 로드합니다.

In [ ]:
data = pd.read_csv(<CODE>)# 결합된 csv 파일을 읽으려면 여기에 코드를 입력하십시오.

처음 5개 레코드를 인쇄합니다.

In [ ]:
# 여기에 코드 입력 

다음은 데이터 집합에 대해 자세히 알아보는 데 도움이 되는 몇 가지 추가 질문입니다.

**질문*8   
1. 데이터 집합에는 행과 열이 몇 개 있나요?   
2. 데이터 집합에는 몇 년이 포함되어 있나요?   
3. 데이터 집합의 날짜 범위는 어떻게 되나요?   
4. 데이터 집합에는 어떤 항공사가 포함되었나요?   
5. 포함된 출발지 공항과 목적지 공항은 어디인가요?

In [ ]:
print("The #rows and #columns are ", <CODE> , " and ", <CODE>)
print("The years in this dataset are: ", list(<CODE>))
print("The months covered in this dataset are: ", sorted(list(<CODE>)))
print("The date range for data is :" , min(<CODE>), " to ", max(<CODE>))
print("The airlines covered in this dataset are: ", list(<CODE>))
print("The Origin airports covered are: ", list(<CODE>))
print("The Destination airports covered are: ", list(<CODE>))

**target column : is_delay**를 정의해 보겠습니다(도착 시간이 15분 넘게 지연되면 1, 그렇지 않으면 0). `rename` 메서드를 사용하여 열 이름을 `ArrDel15`에서 `is_delay`로 바꿉니다.

**힌트**: Pandas `rename` 함수를 사용하면 됩니다([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)).

예시:
```
df.rename(columns={'col1':'column1'}, inplace=True)
```

In [ ]:
data.rename(columns=<CODE>, inplace=True) # 여기에 코드 입력

열에서 null을 찾습니다. `isnull()` 함수를 사용하면 됩니다([설명서](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.isnull.html)).

**힌트**: `isnull()`은 특정 값이 null인지 여부를 감지하고 그 자리에 부울(True 또는 False)을 제공합니다. `sum(axis=0)` 함수를 사용하여 열 수를 합산합니다.

In [ ]:
# 여기에 코드 입력

1658130행 중 22540개(1.3%)에 대한 도착 지연 세부 정보 및 비행 시간이 누락되었습니다. 이러한 행을 제거하거나 대치할 수 있습니다. 설명서에는 누락된 행에 대한 언급이 없습니다.

**힌트**: `~` 연산자를 사용하여 `isnull()` 출력에서 null이 아닌 값을 선택합니다.

예시:
```
null_eg = df_eg[~df_eg['column_name'].isnull()]
```

In [ ]:
### null 열 제거
data = # 여기에 코드 입력

CRSDepTime에서 24시간 형식으로 시간을 가져옵니다.

In [ ]:
data['DepHourofDay'] = # 여기에 코드 입력

## **기계 학습 문제 기술서**
- 일련의 특성을 감안할 때 항공편이 15분 넘게 지연될 것인지 예측할 수 있나요?
- 목표 변수는 0/1 값만 사용하므로 분류 알고리즘을 사용할 수 있습니다. 

모형화를 진행하기 전에 항상 특성 분포, 상관 관계 등을 살펴보는 것이 좋습니다.
- 데이터의 비선형성/패턴을 파악할 수 있습니다.
    - 선형 모형: 거듭제곱/지수/상호 작용 특성 추가
    - 비선형 모형 시도
- 데이터 불균형 
    - 편향된 모형 성능을 제공하지 않는 지표 선택(정확도 대 AUC)
    - 가중치/사용자 지정 손실 함수 사용
- 누락 데이터
    - 평균, 중위수, 최빈값(수치형 변수), 빈도 클래스(범주형 변수)와 같은 단순 통계를 기반으로 대체 수행
    - 군집화 기반 대체(열 값을 예측하는 KNN)
    - 열 삭제

### 데이터 탐색

#### 클래스 지연 대 지연 없음 확인

**힌트**: `groupby` 플롯([설명서](https://matplotlib.org/tutorials/introductory/pyplot.html))을 `bar` 플롯([설명서](https://matplotlib.org/tutorials/introductory/pyplot.html))과 함께 사용하여 클래스의 빈도 대 분포를 표시합니다.

In [ ]:
(data.groupby(<CODE>).size()/len(data) ).plot(kind='bar')# 여기에 코드 입력
plt.ylabel('Frequency')
plt.title('Distribution of classes')
plt.show()

**질문**: 지연 대 지연 없음 비율에 대한 막대 플롯을 통해 무엇을 추론할 수 있나요?

In [ ]:
# 여기에 답변 입력

**질문**: 

- 지연이 가장 많이 발생하는 달은 언제 인가요?
- 하루 중 언제 가장 지연이 많이 발생하나요?
- 일주일 중 언제 가장 지연이 많이 발생하나요?
- 가장 많이 지연되는 항공사는 어디인가요?
- 가장 많이 지연되는 출발지 및 목적지 공항은 어디인가요?
- 비행 거리가 지연에 영향을 미치는 요인인가요?

In [ ]:
viz_columns = ['Month', 'DepHourofDay', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest']
fig, axes = plt.subplots(3, 2, figsize=(20,20), squeeze=False)
# fig.autofmt_xdate(rotation=90)

for idx, column in enumerate(viz_columns):
    ax = axes[idx//2, idx%2]
    temp = data.groupby(column)['is_delay'].value_counts(normalize=True).rename('percentage').\
    mul(100).reset_index().sort_values(column)
    sns.barplot(x=column, y="percentage", hue="is_delay", data=temp, ax=ax)
    plt.ylabel('% delay/no-delay')
    

plt.show()

In [ ]:
sns.lmplot( x="is_delay", y="Distance", data=data, fit_reg=False, hue='is_delay', legend=False)
plt.legend(loc='center')
plt.xlabel('is_delay')
plt.ylabel('Distance')
plt.show()

In [ ]:
# 여기에 답변 입력

### 특성

모든 열과 각 열의 특정 유형을 살펴봅니다.

In [ ]:
data.columns

필요한 열 필터링:
- 날짜를 설명하는 Year, Quarter, Month, DayofMonth 및 DayOfWeek 열이 있으므로 날짜가 중복됩니다.
- OriginState 및 DestState 대신 Origin 및 Dest 코드를 사용합니다.
- 항공편 지연 여부만 분류하는 것이므로 TotalDelayMinutes, DepDelayMinutes 및 ArrDelayMinutes는 필요하지 않습니다.

DepHourofDay는 목표와 정량적 관계가 없으므로 범주형 변수로 취급합니다.
- 이를 One-Hot 인코딩해야 하는 경우 23개의 열이 추가로 생성됩니다.
- 범주형 변수를 처리하는 다른 대안으로는 해시 인코딩, 정규화된 평균 인코딩, 값 버킷화 등이 있습니다.
- 여기에서는 버킷으로 분할하기만 합니다.

**힌트**: 열 유형을 범주로 변경하려면 `astype` 함수([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html))를 사용합니다.

In [ ]:
data_orig = data.copy()
data = data[[ 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay', 'is_delay']]
categorical_columns = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
       'Reporting_Airline', 'Origin', 'Dest', 'DepHourofDay', 'is_delay']
for c in categorical_columns:
    data[c] = data[c].astype('category')# 여기에 코드 입력

One-Hot 인코딩을 사용하려면 위에서 선택한 범주 열에 Pandas `get_dummies` 함수를 사용합니다. 그런 다음 생성된 특성을 Pandas `concat` 함수를 사용하여 원래 데이터 집합에 연결할 수 있습니다. 범주형 변수를 인코딩하는 경우 `drop_first=True` 키워드로 *더미 인코딩*을 수행할 수도 있습니다. 더미 인코딩에 대한 자세한 내용은 https://en.wikiversity.org/wiki/Dummy_variable_(statistics)를 참조하십시오.

예시:
```
pd.get_dummies(df[['column1','columns2']], drop_first=True)
```

In [ ]:
data_dummies = pd.get_dummies(<CODE>, drop_first=True) # 여기에 코드 입력
data = pd.concat([<CODE>, <CODE>], axis = 1)
categorical_columns.remove('is_delay')
data.drop(categorical_columns,axis=1, inplace=True)

데이터 집합의 길이와 새로운 열을 확인합니다.

In [ ]:
# 여기에 코드 입력

In [ ]:
# 여기에 코드 입력

**샘플 답변:** 
```
Index(['Distance', 'is_delay', 'Quarter_2', 'Quarter_3', 'Quarter_4',
       'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7',
       'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
       'DayofMonth_2', 'DayofMonth_3', 'DayofMonth_4', 'DayofMonth_5',
       'DayofMonth_6', 'DayofMonth_7', 'DayofMonth_8', 'DayofMonth_9',
       'DayofMonth_10', 'DayofMonth_11', 'DayofMonth_12', 'DayofMonth_13',
       'DayofMonth_14', 'DayofMonth_15', 'DayofMonth_16', 'DayofMonth_17',
       'DayofMonth_18', 'DayofMonth_19', 'DayofMonth_20', 'DayofMonth_21',
       'DayofMonth_22', 'DayofMonth_23', 'DayofMonth_24', 'DayofMonth_25',
       'DayofMonth_26', 'DayofMonth_27', 'DayofMonth_28', 'DayofMonth_29',
       'DayofMonth_30', 'DayofMonth_31', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'Reporting_Airline_DL', 'Reporting_Airline_OO', 'Reporting_Airline_UA',
       'Reporting_Airline_WN', 'Origin_CLT', 'Origin_DEN', 'Origin_DFW',
       'Origin_IAH', 'Origin_LAX', 'Origin_ORD', 'Origin_PHX', 'Origin_SFO',
       'Dest_CLT', 'Dest_DEN', 'Dest_DFW', 'Dest_IAH', 'Dest_LAX', 'Dest_ORD',
       'Dest_PHX', 'Dest_SFO'],
      dtype='object')
```

이제 모형 훈련을 수행할 준비가 되었습니다. 데이터를 분할하기 전에 `is_delay` 열의 이름을 `target`으로 변경합니다.

**힌트**: Pandas `rename` 함수를 사용하면 됩니다([설명서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)).

In [ ]:
data.rename(columns = {<CODE>:<CODE>}, inplace=True )# 여기에 코드 입력

## <span style="color:red"> 실습 2 종료 </span>

프로젝트 파일을 로컬 컴퓨터에 저장합니다. 방법은 다음과 같습니다.

1. 페이지 상단에 있는 **파일** 메뉴를 클릭합니다. 

1. **다운로드**를 선택하고 **Notebook(.ipynb)**을 클릭합니다.  

그러면 현재 노트북이 컴퓨터의 기본 다운로드 폴더로 다운로드됩니다.

# 3단계: 모형 훈련 및 평가

데이터 프레임에서 기계 학습 알고리즘이 사용할 수 있는 형식으로 데이터 집합을 변환할 때 포함해야 하는 몇 가지 예비 단계가 있습니다. Amazon SageMaker의 경우 다음 단계를 수행해야 합니다.

1. `sklearn.model_selection.train_test_split`을 사용하여 데이터를 `train_data`, `validation_data` 및 `test_data`로 분할합니다.  
2. 데이터 집합을 Amazon SageMaker 훈련 작업에서 사용할 수 있는 적절한 파일 형식으로 변환합니다. CSV 파일 또는 레코드 protobuf로 변환하면 됩니다. 자세한 내용은 [훈련을 위한 공통 데이터 형식](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html) 페이지를 참조하십시오.  
3. 데이터를 Amazon S3 버킷에 업로드합니다. 이전에 버킷을 생성한 적이 없는 경우 [버킷 생성](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html) 페이지를 참조하십시오.  

다음 셀을 사용하여 이러한 단계를 완료합니다. 필요에 따라 셀을 삽입하고 삭제합니다.

### <span style="color: blue;">프로젝트 프레젠테이션: 이 단계에서 내린 주요 의사 결정을 프로젝트 프레젠테이션에 기록해 둡니다.</span>

### 훈련 집합 분할

In [ ]:
from sklearn.model_selection import train_test_split

def create_training_sets(data):
    """
    데이터 프레임을 훈련, 검증 및 검정으로 변환
    params:
        data: 분할할 데이터 집합이 있는 데이터 프레임
    Returns:
        train_features: 훈련 특성 데이터 집합
        test_features: 검정 특성 데이터 집합 
        train_labels: 훈련 데이터 집합의 레이블
        test_labels: 검정 데이터 집합의 레이블
        val_features: 검증 특성 데이터 집합
        val_labels: 검증 데이터 집합의 레이블
    """
    # 데이터 프레임에서 목표 변수를 추출하고 유형을 float32로 변환
    ys = np.array(<CODE>).astype("float32") # 여기에 코드 입력
    
    # 목표 열을 비롯하여 원하지 않는 모든 열 삭제
    drop_list = # 여기에 코드 입력
    
    # drop_list에서 열을 삭제하고 데이터를 유형 float32의 NumPy 배열로 변환
    xs = np.array(data.drop(<CODE>, axis=1)).astype("float32")# 여기에 코드 입력
    
    np.random.seed(0)

    # sklearn 함수인 train_test_split을 사용하여 데이터 집합을 훈련 80%와 검정 20% 비율로 분할합니다.
    # 예: train_test_split(x, y, test_size=0.3)
    train_features, test_features, train_labels, test_labels = # 여기에 코드 입력
    
    # sklearn 함수를 다시 사용하여 검정 데이터 집합을 검증 50%와 검정 50%로 분할
    val_features, test_features, val_labels, test_labels = # 여기에 코드 입력
    
    
    return train_features, test_features, train_labels, test_labels, val_features, val_labels

In [ ]:
# 다음 함수를 사용하여 데이터 집합 생성
train_features, test_features, train_labels, test_labels, val_features, val_labels = create_training_sets(data)

print(f"Length of train_features is: {<CODE>}")
print(f"Length of train_labels is: {<CODE>}")
print(f"Length of val_features is: {<CODE>}")
print(f"Length of val_labels is: {<CODE>}")
print(f"Length of test_features is: {<CODE>}")
print(f"Length of test_labels is: {<CODE>}")

**샘플 답변**
```
Length of train_features is: (1308472, 71)
Length of train_labels is: (1308472,)
Length of val_features is: (163559, 71)
Length of val_labels is: (163559,)
Length of test_features is: (163559, 71)
Length of test_labels is: (163559,)
```

### 기준 분류 모형

In [ ]:
import sagemaker
from sagemaker.predictor import csv_serializer
from sagemaker.amazon.amazon_estimator import RecordSet
import boto3

num_classes = # 여기에 코드 입력

# ml.m4.xlarge 1개로 LinearLearner 추정기 객체 인스턴스화
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=<CODE>,
                                               train_instance_type=<CODE>,
                                               predictor_type=<CODE>,
                                              binary_classifier_model_selection_criteria=<CODE>)

### 샘플 코드
```
num_classes = len(pd.unique(train_labels))
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                              train_instance_count=1,
                                              train_instance_type='ml.m4.xlarge',
                                              predictor_type='binary_classifier',
                                             binary_classifier_model_selection_criteria = 'cross_entropy_loss')
                                              
```

선형 학습자는 protobuf 또는 CSV 콘텐츠 유형의 훈련 데이터를 수용하고 protobuf, CSV 또는 JSON 콘텐츠 유형의 추론 요청을 수용합니다. 훈련 데이터에는 특성과 ground-truth 레이블이 있으며, 추론 요청의 데이터에는 특성만 있습니다.

프로덕션 파이프라인에서는 데이터를 Amazon SageMaker protobuf 형식으로 변환하여 Amazon S3에 저장하는 것이 좋습니다. 하지만 빠르게 시작하고 실행할 수 있도록 AWS에서는 데이터 집합이 로컬 메모리에 적합할 정도로 작을 때 쉽게 변환하고 업로드할 수 있는 편리한 메서드인 `record_set`을 제공합니다. 이 메서드는 이미 가지고 있는 것과 같은 NumPy 배열을 수락하므로 여기에서 사용해 보겠습니다. '`RecordSet` 객체는 데이터의 임시 Amazon S3 위치를 추적합니다. `estimator.record_set` 함수를 사용하여 train, validation 및 test 레코드를 생성합니다. 그런 다음 `estimator.fit` 함수를 사용하여 훈련 작업을 시작합니다.

In [ ]:
### train, val, test 레코드 생성
train_records = linear.record_set(<CODE>,<CODE>, channel='train')# 여기에 코드 입력
val_records = linear.record_set(<CODE>,<CODE>, channel='validation')# 여기에 코드 입력
test_records = linear.record_set(<CODE>,<CODE>, channel='test')# 여기에 코드 입력

이제 방금 업로드한 데이터 집합으로 모형을 훈련합니다.

### 샘플 코드
```
linear.fit([train_records,val_records,test_records])
```

In [ ]:
### 분류자 피팅
# 여기에 코드 입력

## 모형 평가
이 섹션에서는 훈련된 모형을 평가합니다. 먼저 `estimator.deploy` 함수를 `initial_instance_count= 1` 및 `instance_type= 'ml.m4.xlarge'`와 함께 사용하여 Amazon SageMaker에 모형을 배포합니다.

In [ ]:
### 배치 예측용 엔드포인트 배포
classifier_predictor = classifier_estimator.deploy(initial_instance_count=<CODE>, 
                                                   instance_type=<CODE>) # 여기에 코드 입력

엔드포인트가 'InService' 상태가 되면 검정 집합에서 모형의 성능이 어떤지 평가합니다. `predict_batches` 함수를 사용하여 검정 집합의 지표를 예측합니다.

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support

def predict_batches(predictor, features, labels):
    """
    평가 결과 반환
    predictor: 모형의 예측 변수 객체
    features: 모형의 입력 특성
    label: Ground Truth 목표 값
    """
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(features, 100)]

    # protobuf 응답을 구문 분석하여 예측된 레이블 추출
    extract_label = lambda x: x.label['predicted_label'].float32_tensor.values
    preds = np.concatenate([np.array([extract_label(x) for x in batch]) for batch in prediction_batches])
    preds = preds.reshape((-1,))

    # 정확도 계산
    accuracy = (preds == labels).sum() / labels.shape[0]
    print(f'Accuracy: {accuracy}')
    
    auc = roc_auc_score(labels, preds)
    print(f'AUC : {auc}')
    
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, preds, average = 'binary')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1_score: {f1_score}')
    
    confusion_matrix = pd.crosstab(index=labels, columns=np.round(preds), rownames=['True'], colnames=['predictions']).astype(int)
    plt.figure(figsize = (5,5))
    sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix') 

혼동 행렬을 보려면 검정 데이터 집합에서 `predict_batches` 함수를 실행합니다.

In [ ]:
# 여기에 코드 입력

### 고려해야 할 주요 질문:
1. 훈련 집합과 비교하여 검정 집합에서의 모형 성능이 어떤가요? 이 비교에서 무엇을 추론할 수 있나요? 

2. 정확도, 정밀도, 재현율과 같은 지표의 결과 간에 분명한 차이가 있나요? 만약 그렇다면, 이러한 차이가 나타나는 이유는 무엇일까요? 

3. 비즈니스 상황과 목표를 고려래 볼 때 여기서 고려해야 할 가장 중요한 지표는 무엇일까요? 그 이유는 무엇일까요?

4. 가장 중요하다고 생각하는 지표의 결과가 비즈니스 관점에서 필요한 결과로도 충분한가요? 그렇지 않다면, 다음 반복(다음에 나오는 특성 엔지니어링 섹션)에서 무엇을 변경할 수 있을까요? 

아래 셀을 사용하여 이러한 질문과 기타 질문에 답하십시오. 필요에 따라 셀을 삽입하고 삭제합니다.

### <span style="color: blue;">프로젝트 프레젠테이션: 이러한 질문과 이 섹션에서 대답할 수 있는 유사한 질문에 대한 답변을 프로젝트 프레젠테이션에 기록합니다. 주요 세부 정보와 결정한 내용을 프로젝트 프레젠테이션에 기록합니다.</span>


**질문**: 혼동 행렬에서 요약할 수 있는 것은 무엇인가요?


In [ ]:
# 여기에 답변 입력

## <span style="color:red"> 실습 3 종료 </span>

프로젝트 파일을 로컬 컴퓨터에 저장합니다. 방법은 다음과 같습니다.

1. 페이지 상단에 있는 **파일** 메뉴를 클릭합니다. 

1. **다운로드**를 선택하고 **Notebook(.ipynb)**을 클릭합니다.  

그러면 현재 노트북이 컴퓨터의 기본 다운로드 폴더로 다운로드됩니다.

# 반복 II

# 4단계: 특성 엔지니어링

이제 모형을 훈련하고 평가하는 과정을 한 번 반복했습니다. 처음 모형을 사용해 도달한 결과로는 비즈니스 문제를 해결하는 데 충분하지 않을 수 있다는 점을 감안할 때 모형 성능을 개선하기 위해 데이터에 대해 변경할 수 있는 사항은 무엇인가요?

### 고려해야 할 주요 질문:
1. 두 개의 주요 클래스(지연 및 지연 아님)의 균형이 모형 성능에 어떤 영향을 미치나요?
2. 상관 관계가 있는 특성이 있나요?
3. 이 단계에서 모형 성능에 긍정적인 영향을 미칠 수 있는 특성 축소 기법이 있나요? 
4. 데이터/데이터 집합을 더 추가하는 것이 좋을까요?
4. 약간의 특성 엔지니어링을 수행한 후 첫 번째 반복과 비교하여 모형 성능이 어떤가요?

아래 셀을 사용하여 모형 성능을 향상할 수 있다고 생각되는 특정 특성 엔지니어링 기법(위의 질문에 따라)을 수행합니다. 필요에 따라 셀을 삽입하고 삭제합니다.

### <span style="color: blue;">프로젝트 프레젠테이션: 이 섹션에서 사용하는 주요 결정 사항과 기법은 물론 모형을 다시 평가한 후 얻은 새로운 성능 지표를 프로젝트 프레젠테이션에 기록합니다.</span>

시작하기 전에 왜 정확도 및 재현율은 약 80%인데 정확도는 99%인지 생각해 보십시오.

#### 더 많은 특성 추가

1. Holidays
2. Weather

2014년부터 2018까지의 공휴일 목록이 알려져 있기 때문에 표시 변수 **is_holiday**를 생성하여 이를 표시할 수 있습니다.
다른 날보다 공휴일에 비행기 지연율이 더 높을 수 있다는 가설입니다. 2014녀~2018년의 공휴일이 포함된 부울 변수 `is_holiday`를 추가합니다.

In [ ]:
# 출처: http://www.calendarpedia.com/holidays/federal-holidays-2014.html

holidays_14 = ['2014-01-01', '2014-01-20', '2014-02-17', '2014-05-26', '2014-07-04', '2014-09-01', '2014-10-13', '2014-11-11', '2014-11-27', '2014-12-25' ] 
holidays_15 = ['2015-01-01', '2015-01-19', '2015-02-16', '2015-05-25', '2015-06-03', '2015-07-04', '2015-09-07', '2015-10-12', '2015-11-11', '2015-11-26', '2015-12-25'] 
holidays_16 = ['2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30', '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25', '2016-12-26']
holidays_17 = ['2017-01-02', '2017-01-16', '2017-02-20', '2017-05-29' , '2017-07-04', '2017-09-04' ,'2017-10-09', '2017-11-10', '2017-11-23', '2017-12-25']
holidays_18 = ['2018-01-01', '2018-01-15', '2018-02-19', '2018-05-28' , '2018-07-04', '2018-09-03' ,'2018-10-08', '2018-11-12','2018-11-22', '2018-12-25']
# holidays_19 = ['2019-01-01', '2019-01-21', '2019-02-18', '2019-05-27' , '2019-07-04', '2019-09-02' ,'2019-10-14', '2019-11-11','2019-11-28', '2019-12-25']
holidays = holidays_14+ holidays_15+ holidays_16 + holidays_17+ holidays_18

### 공휴일에 대한 표시 변수 추가
data_orig['is_holiday'] = # 여기에 코드 입력 

기상 데이터는 https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&amp;stations=USW00023174,USW00012960,USW00003017,USW00094846,USW00013874,USW00023234,USW00003927,USW00023183,USW00013881&amp;dataTypes=AWND,PRCP,SNOW,SNWD,TAVG,TMIN,TMAX&amp;startDate=2014-01-01&amp;endDate=2018-12-31에서 가져왔습니다.
<br>

이 데이터 집합에는 공항 코드별로 도시의 풍속, 강수, 강설 및 온도에 대한 정보가 있습니다.

**질문**: 비, 폭풍 또는 눈으로 인한 악천후로 비행기가 지연될 수 있나요? 확인해 보겠습니다!

In [ ]:
%%bash

aws s3 cp s3://aws-tc-largeobjects/ILT-TF-200-MLDWTS/flight_delay_project/daily-summaries.csv /home/ec2-user/SageMaker/project/data/
#wget 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&amp;stations=USW00023174,USW00012960,USW00003017,USW00094846,USW00013874,USW00023234,USW00003927,USW00023183,USW00013881&amp;dataTypes=AWND,PRCP,SNOW,SNWD,TAVG,TMIN,TMAX&amp;startDate=2014-01-01&amp;endDate=2018-12-31' -O /home/ec2-user/SageMaker/project/data/daily-summaries.csv

데이터 집합에 공항 코드별로 준비된 기상 데이터를 가져옵니다. 아래의 관측소와 공항을 분석에 사용하고 기상 관측소를 공항 이름에 매핑하는 `airport`라는 새로운 열을 생성합니다.

In [ ]:
weather = pd.read_csv(<CODE>) # 'daily-summaries.csv' 파일을 읽으려면 여기에 코드 입력
station = ['USW00023174','USW00012960','USW00003017','USW00094846',
           'USW00013874','USW00023234','USW00003927','USW00023183','USW00013881'] 
airports = ['LAX', 'IAH', 'DEN', 'ORD', 'ATL', 'SFO', 'DFW', 'PHX', 'CLT']

### 기상 관측소를 공항 코드에 매핑
station_map = # 여기에 코드 입력 
weather['airport'] = # 여기에 코드 입력 

`DATE` 열에서 `MONTH`라는 다른 열을 생성합니다.

In [ ]:
weather['MONTH'] = weather[<CODE>].apply(lambda x: x.split('-')[1])# 여기에 코드 입력 
weather.head()

### 샘플 출력
```
  STATION DATE AWND PRCP SNOW SNWD TAVG TMAX TMIN airport MONTH
0 USW00023174 2014-01-01 16 0 NaN NaN 131.0 178.0 78.0 LAX 01
1 USW00023174 2014-01-02 22 0 NaN NaN 159.0 256.0 100.0 LAX 01
2 USW00023174 2014-01-03 17 0 NaN NaN 140.0 178.0 83.0 LAX 01
3 USW00023174 2014-01-04 18 0 NaN NaN 136.0 183.0 100.0 LAX 01
4 USW00023174 2014-01-05 18 0 NaN NaN 151.0 244.0 83.0 LAX 01
```

`fillna()`를 사용하여 `SNOW` 및 `SNWD` 열의 결측치를 분석하고 처리합니다. `isna()` 함수를 사용하여 모든 열의 결측치를 확인합니다.

In [ ]:
weather.SNOW.fillna(<CODE>, inplace=True)# 여기에 코드 입력
weather.SNWD.fillna(<CODE>, inplace=True)# 여기에 코드 입력
weather.isna().sum()

**질문**: TAVG, TMAX, TMIN에 대한 결측치가 있는 행의 인덱스를 인쇄합니다.

**힌트**: `isna()` 함수를 사용하여 누락된 행을 찾은 다음 idx 변수의 목록을 사용하여 인덱스를 가져옵니다.

In [ ]:
idx = np.array([i for i in range(len(weather))])
TAVG_idx = # 여기에 코드 입력 
TMAX_idx = # 여기에 코드 입력 
TMIN_idx = # 여기에 코드 입력 
TAVG_idx

### 샘플 출력

```
array([ 3956, 3957, 3958, 3959, 3960, 3961, 3962, 3963, 3964,
        3965, 3966, 3967, 3968, 3969, 3970, 3971, 3972, 3973,
        3974, 3975, 3976, 3977, 3978, 3979, 3980, 3981, 3982,
        3983, 3984, 3985, 4017, 4018, 4019, 4020, 4021, 4022,
        4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031,
        4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040,
        4041, 4042, 4043, 4044, 4045, 4046, 4047, 13420])
```

누락된 TAVG, TMAX 및 TMIN을 특정 관측소/공항의 평균값으로 대체할 수 있습니다. TAVG_idx의 연속 행이 누락되었기 때문에 이전 값으로 대체하는 것은 불가능합니다. 대신 평균값으로 대체합니다. `groupby` 함수를 사용하여 평균값으로 변수를 집계합니다.

In [ ]:
weather_impute = weather.groupby([<CODE>]).agg({'TAVG':'mean','TMAX':'mean', 'TMIN':'mean' }).reset_index()# 여기에 코드 입력
weather_impute.head(2)

평균 데이터를 기상 데이터와 병합합니다.

In [ ]:
### 어제 데이터 가져오기
weather = pd.merge(weather, weather_impute, how='left', left_on=['MONTH','STATION'], right_on = ['MONTH','STATION'])\
.rename(columns = {'TAVG_y':'TAVG_AVG',
                   'TMAX_y':'TMAX_AVG', 
                   'TMIN_y':'TMIN_AVG',
                   'TAVG_x':'TAVG',
                   'TMAX_x':'TMAX', 
                   'TMIN_x':'TMIN'})

결측치가 있는지 다시 확인합니다.

In [ ]:
weather.TAVG[TAVG_idx] = weather.TAVG_AVG[TAVG_idx]
weather.TMAX[TMAX_idx] = weather.TMAX_AVG[TMAX_idx]
weather.TMIN[TMIN_idx] = weather.TMIN_AVG[TMIN_idx]
weather.isna().sum()

데이터 집합에서 `STATION,MONTH,TAVG_AVG,TMAX_AVG,TMIN_AVG,TMAX,TMIN,SNWD`를 삭제합니다.

In [ ]:
weather.drop(columns=['STATION','MONTH','TAVG_AVG', 'TMAX_AVG', 'TMIN_AVG', 'TMAX' ,'TMIN', 'SNWD'],inplace=True)

출발지 및 목적지 기상 상태를 데이터 집합에 추가합니다.

In [ ]:
### 출발지 기상 상태 추가
data_orig = pd.merge(data_orig, weather, how='left', left_on=['FlightDate','Origin'], right_on = ['DATE','airport'])\
.rename(columns = {'AWND':'AWND_O','PRCP':'PRCP_O', 'TAVG':'TAVG_O', 'SNOW': 'SNOW_O'})\
.drop(columns=['DATE','airport'])

### 목적지 기상 상태 추가
data_orig = pd.merge(data_orig, weather, how='left', left_on=['FlightDate','Dest'], right_on = ['DATE','airport'])\
.rename(columns = {'AWND':'AWND_D','PRCP':'PRCP_D', 'TAVG':'TAVG_D', 'SNOW': 'SNOW_D'})\
.drop(columns=['DATE','airport'])

**참고**: 조인 후에는 null/NA를 확인하는 것이 좋습니다.

In [ ]:
sum(data.isna().any())

In [ ]:
data_orig.columns

One-Hot 인코딩을 사용하여 범주형 데이터를 수치형 데이터로 변환합니다.

In [ ]:
data = data_orig.copy()
data = data[['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay', 'is_delay', 'is_holiday', 'AWND_O', 'PRCP_O',
       'TAVG_O', 'AWND_D', 'PRCP_D', 'TAVG_D', 'SNOW_O', 'SNOW_D']]


categorical_columns = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 
       'Reporting_Airline', 'Origin', 'Dest', 'is_holiday','is_delay']
for c in categorical_columns:
    data[c] = data[c].astype('category')

In [ ]:
data_dummies = # 여기에 코드 입력

### 샘플 코드

```
data_dummies = pd.get_dummies(data[['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest', 'is_holiday']], drop_first=True)
data = pd.concat([data, data_dummies], axis = 1)
categorical_columns.remove('is_delay')
data.drop(categorical_columns,axis=1, inplace=True)
```

새로운 열을 확인합니다.

In [ ]:
data.columns

### 샘플 출력

```
Index(['Distance', 'DepHourofDay', 'is_delay', 'AWND_O', 'PRCP_O', 'TAVG_O',
       'AWND_D', 'PRCP_D', 'TAVG_D', 'SNOW_O', 'SNOW_D', 'Year_2015',
       'Year_2016', 'Year_2017', 'Year_2018', 'Quarter_2', 'Quarter_3',
       'Quarter_4', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
       'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
       'DayofMonth_2', 'DayofMonth_3', 'DayofMonth_4', 'DayofMonth_5',
       'DayofMonth_6', 'DayofMonth_7', 'DayofMonth_8', 'DayofMonth_9',
       'DayofMonth_10', 'DayofMonth_11', 'DayofMonth_12', 'DayofMonth_13',
       'DayofMonth_14', 'DayofMonth_15', 'DayofMonth_16', 'DayofMonth_17',
       'DayofMonth_18', 'DayofMonth_19', 'DayofMonth_20', 'DayofMonth_21',
       'DayofMonth_22', 'DayofMonth_23', 'DayofMonth_24', 'DayofMonth_25',
       'DayofMonth_26', 'DayofMonth_27', 'DayofMonth_28', 'DayofMonth_29',
       'DayofMonth_30', 'DayofMonth_31', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'Reporting_Airline_DL', 'Reporting_Airline_OO', 'Reporting_Airline_UA',
       'Reporting_Airline_WN', 'Origin_CLT', 'Origin_DEN', 'Origin_DFW',
       'Origin_IAH', 'Origin_LAX', 'Origin_ORD', 'Origin_PHX', 'Origin_SFO',
       'Dest_CLT', 'Dest_DEN', 'Dest_DFW', 'Dest_IAH', 'Dest_LAX', 'Dest_ORD',
       'Dest_PHX', 'Dest_SFO', 'is_holiday_1'],
      dtype='object')
```

`is_delay` 열의 이름을 다시 `target`으로 바꿉니다. 이전과 동일한 코드를 사용합니다.

In [ ]:
data.rename(columns = {<CODE>:<CODE>}, inplace=True )# 여기에 코드 입력

훈련 집합을 다시 생성합니다.

In [ ]:
# 여기에 코드 입력

### 새로운 기준 분류자

이제 이러한 새로운 특성으로 모형의 예측 성능이 향상되는지 확인합니다.

In [ ]:
# LinearLearner 추정기 객체 인스턴스화
num_classes = # 여기에 코드 입력
classifier_estimator = # 여기에 코드 입력

### 샘플 코드

```
num_classes = len(pd.unique(train_labels)) 
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               train_instance_count=1,
                                               train_instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                              binary_classifier_model_selection_criteria = 'cross_entropy_loss')
```

In [ ]:
train_records = classifier_estimator.record_set(train_features, train_labels, channel='train')
val_records = classifier_estimator.record_set(val_features, val_labels, channel='validation')
test_records = classifier_estimator.record_set(test_features, test_labels, channel='test')

classifier_estimator.fit([train_records, val_records, test_records])

In [ ]:
classifier_predictor = # 여기에 코드 입력

In [ ]:
predict_batches(classifier_predictor, test_features, test_labels)

선형 모형은 성능이 약간 향상되었을 뿐입니다. Amazon SageMaker에서 XGBoost라는 트리 기반 앙상블 모형을 사용해 보십시오.

### XGBoost 모형 사용해 보기

다음은 수행해야 할 단계입니다.  

1. 훈련 집합 변수를 사용하여 train.csv, validation.csv 및 test.csv 등의 CSV 파일로 저장합니다.  
2. 변수에 버킷 이름을 저장합니다. Amazon S3 버킷 이름은 실습 지침의 왼쪽에 나와 있습니다.  
a. `bucket = <LabBucketName>`  
b. `prefix = 'sagemaker/xgboost'`  
3. Boto3를 사용하여 모형을 버킷에 업로드합니다.    

In [ ]:
train_data, validation_data, test_data = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(data)), int(0.9*len(data))])  

pd.concat([train_data['target'], train_data.drop(['target'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['target'], validation_data.drop(['target'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)
pd.concat([test_data['target'], test_data.drop(['target'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

***`<LabBucketName>`**을(를) 실습 계정에 제공된 리소스 이름으로 바꿉니다.

In [ ]:
# 버킷 및 접두사를 정의하려면 여기에 코드 입력
bucket = <LabBucketName> # 여기에 코드 입력
prefix = 'sagemaker/xgboost'

In [ ]:
### S3에 데이터 업로드
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

`sagemaker.s3_input` 함수를 사용하여 훈련 및 검증 데이터 집합에 대한 record_set을 생성합니다.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role = sagemaker.get_execution_role(), 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric = "auc", 
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

새 모형에 대한 예측 변수를 배포하고 검정 데이터 집합으로 모형을 평가합니다.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = <CODE>, 
                           instance_type = <CODE>)# 여기에 코드 입력

In [ ]:
test_data = pd.concat([test_data['target'], test_data.drop(['target'], axis=1)], axis=1)

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None


def predict(predictor , features, labels , prob_threshold = 0.5, rows=500):
    """
    평가 결과 반환
    predictor: 모형의 예측 변수 객체
    features: 모형의 입력 특성
    label: Ground Truth 목표 값
    prob_threshold: 긍정 클래스와 부정 클래스를 구분할 확률 절사점
    """
    split_array = np.array_split(features, int(features.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    preds = np.fromstring(predictions[1:], sep=',')
    preds = preds.reshape((-1,))
    predictions = np.where(preds > prob_threshold , 1, 0)
    labels = labels.reshape((-1,))
    
    
    # 정확도 계산
    accuracy = (predictions == labels).sum() / labels.shape[0]
    print(f'Accuracy: {accuracy}')
    
    auc = roc_auc_score(labels, preds)
    print(f'AUC : {auc}')
    
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, predictions, average = 'binary')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1_score: {f1_score}')
    
    
    confusion_matrix = pd.crosstab(index=labels, columns=np.round(preds), rownames=['True'], colnames=['predictions']).astype(int)
    plt.figure(figsize = (5,5))
    sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix') 
    return list(preds)

In [ ]:
predictions = predict(xgb_predictor, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.5)

### 다른 임계값 시도

In [ ]:
predictions = predict(xgb_predictor, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.8)

**질문**: 검정 집합에서 보여준 모형의 성능에서 어떤 결론을 내릴 수 있을까요?

In [ ]:
# 여기에 답변 입력

### 하이퍼파라미터 최적화(HPO)

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

### 여러 인스턴스를 시작하여 하이퍼파라미터 최적화를 병렬로 수행할 수 있습니다.

xgb = sagemaker.estimator.Estimator(container,
                                    role=sagemaker.get_execution_role(), 
                                    train_instance_count= 2, # 이러한 인스턴스에 대해 한도 설정 필요
                                    train_instance_type='ml.m4.xlarge', 
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(eval_metric='auc',
                        objective='binary:logistic',
                        num_round=100,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)

In [ ]:
hyperparameter_ranges = {'eta': ContinuousParameter(0, 0.5),
                        'min_child_weight': ContinuousParameter(3, 10),
                         'num_round': IntegerParameter(10, 150),
                        'alpha': ContinuousParameter(0, 2),
                        'max_depth': IntegerParameter(10, 15)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10,
                            max_parallel_jobs=3)

In [ ]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

<i class="fas fa-exclamation-triangle" style="color:red"></i>훈련 작업이 완료될 때까지 기다립니다. 25~30분이 걸릴 수 있습니다.

**하이퍼파라미터 최적화 작업을 모니터링하려면,**  

1. AWS Management Console의 **Services** 메뉴에서 **Amazon SageMaker**를 클릭합니다.  
1. **훈련** > **하이퍼파라미터 튜닝 작업**을 클릭합니다.  
1. 각 하이퍼파라미터 튜닝 작업의 상태, 목표 지표 값 및 로그를 확인할 수 있습니다.  

In [ ]:
sage_client = boto3.Session().client('sagemaker')
tuning_job_name = tuner.latest_tuning_job.job_name

# 다음 셀을 실행하여 하이퍼파라미터 튜닝 작업의 현재 상태 확인
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']


from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")
    
### 데이터 프레임에서 하이퍼파라미터 튜닝 작업 결과 가져오기
tuner_df = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name).dataframe()

if len(tuner_df) > 0:
    df = tuner_df[tuner_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1) # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
tuner_df

하이퍼파라미터 최적화 훈련에서 최적의 모형을 배포합니다.

In [ ]:
xgb_predictor_hpo = # 여기에 코드 입력 

In [ ]:
xgb_predictor_hpo.content_type = 'text/csv'
xgb_predictor_hpo.serializer = csv_serializer
xgb_predictor_hpo.deserializer = None

predictions = predict(xgb_predictor_hpo, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.5)

**질문**: 다른 하이퍼파라미터와 하이퍼파라미터 범위를 시도합니다. 이렇게 하면 모형이 개선되나요?

## 특성 중요도

모든 하이퍼파라미터 튜닝 작업의 모형 파일은 위에 있는 표의 "{bucket}/{prefix}/output/" 폴더에 있는 것처럼 훈련 작업 이름으로 저장됩니다. 모형을 로드하여 일반 sklearn 모형 객체처럼 해석에 사용할 수 있습니다.

In [ ]:
#best_hpo_model_path = "s3://" + bucket + "/sagemaker/xgboost/output/<Best model TrainingJobName>/output/model.tar.gz"
best_hpo_model_path = <path-to-your-model>
### best_hpo_model_path를 로컬에 다운로드
!aws s3 cp {best_hpo_model_path} .

In [ ]:
# xgboost 설치
!pip install xgboost=='0.90'

#### 선택한 모형 파일 로드

In [ ]:
import pickle
import tarfile
import xgboost

with open('model.tar.gz', 'rb') as f:
    with tarfile.open(fileobj=f, mode='r') as tar_f:
        with tar_f.extractfile('xgboost-model') as extracted_f:
            xgbooster = pickle.load(extracted_f)

#### 열 이름을 XGBoost 모형에 매핑

In [ ]:
columns = list(data.columns)
columns.remove('target')

In [ ]:
feature_importance = xgbooster.get_fscore()
feature_importance_col = {}

for column, fname in zip(columns, xgbooster.feature_names):
    try:
         feature_importance_col[column] = feature_importance[fname]
    except Exception:
        pass

#### 특성 중요도 값을 기준으로 정렬

In [ ]:
sorted(feature_importance_col.items(), key=lambda kv: kv[1], reverse=True)

### 샘플 출력

```
[('AWND_O', 13851),
 ('AWND_D', 13452),
 ('DepHourofDay', 13344),
 ('TAVG_O', 13106),
 ('TAVG_D', 12800),
 ('Distance', 8478),
 ('PRCP_O', 4210),
 ('PRCP_D', 3916),
 ('Reporting_Airline_UA', 1791),
 ('Year_2016', 1290),
 ('Year_2015', 1285),
 ('Year_2018', 1157),
 ('Quarter_4', 1092),
 ('Year_2017', 1009),
 ('DayOfWeek_5', 838),
 ('DayOfWeek_4', 833),
 ('Quarter_2', 799),
 ('DayOfWeek_7', 783),
 ('Reporting_Airline_WN', 736),
 ('DayOfWeek_3', 718),
 ('Origin_ORD', 706),
 ('DayOfWeek_2', 685),
 ('Reporting_Airline_DL', 663),
 ('Dest_LAX', 627),
 ('DayOfWeek_6', 614),
 ('Reporting_Airline_OO', 596),
 ('Origin_LAX', 590),
 ('Month_11', 565),
 ('Month_5', 543),
 ('Dest_ORD', 539),
 ('SNOW_O', 506),
 ('Month_8', 497),
 ('Month_12', 495),
 ('Month_7', 484),
 ('Origin_DFW', 480),
 ('Quarter_3', 477),
 ('Dest_DFW', 462),
 ('Origin_DEN', 456),
 ('Origin_SFO', 438),
 ('Month_6', 437),
 ('Dest_SFO', 437),
 ('Month_10', 422),
 ('Month_9', 407),
 ('Month_4', 394),
 ('Dest_DEN', 379),
 ('SNOW_D', 375),
 ('Month_3', 369),
 ('Dest_IAH', 369),
 ('Origin_PHX', 347),
 ('Dest_PHX', 346),
 ('Origin_IAH', 342),
 ('DayofMonth_15', 324),
 ('DayofMonth_17', 322),
 ('DayofMonth_21', 322),
 ('Origin_CLT', 322),
 ('DayofMonth_20', 316),
 ('DayofMonth_9', 313),
 ('DayofMonth_19', 307),
 ('DayofMonth_26', 303),
 ('DayofMonth_7', 301),
 ('DayofMonth_4', 300),
 ('DayofMonth_2', 299),
 ('DayofMonth_27', 298),
 ('DayofMonth_3', 294),
 ('DayofMonth_28', 287),
 ('Month_2', 284),
 ('DayofMonth_5', 284),
 ('DayofMonth_12', 282),
 ('DayofMonth_11', 276),
 ('Dest_CLT', 276),
 ('DayofMonth_6', 275),
 ('DayofMonth_22', 274),
 ('DayofMonth_8', 273),
 ('DayofMonth_18', 273),
 ('DayofMonth_29', 267),
 ('DayofMonth_23', 266),
 ('DayofMonth_14', 263),
 ('DayofMonth_30', 260),
 ('DayofMonth_10', 259),
 ('DayofMonth_16', 257),
 ('DayofMonth_24', 255),
 ('DayofMonth_13', 245),
 ('DayofMonth_25', 237),
 ('is_holiday_1', 231),
 ('DayofMonth_31', 164)]
```

위의 특성 중요도를 바탕으로, 출발지와 목적지 모두에서 DepHourofDay, Airwind 및 Temperature가 항공편 지연을 결정하는 데 중요한 영향을 미치는 요인임을 알 수 있습니다.

이는 특성 면에서 직관적인 것이 무엇이고 모형이 실제로 학습하는 것이 무엇인지를 검증하는 한 가지 방법입니다.

## 결론

이제 모형을 훈련하고 평가하는 과정을 최소한 두 번 반복했습니다. 이제 이 프로젝트를 마무리하고 학습한 내용과 앞으로 수행할 단계에 대해 생각해 볼 차례입니다(시간 여유가 있다고 가정할 때). 아래 셀을 사용하여 이러한 질문 및 기타 관련 질문에 답합니다.

1. 모형 성능이 비즈니스 목표에 부합하나요? 그렇지 않은 경우 튜닝할 시간이 더 있었다면 어떻게 다르게 할 수 있을까요?
2. 데이터 집합, 특성 및 하이퍼파라미터를 변경함에 따라 모형이 어느 정도 향상되었나요? 이 프로젝트 전체에서 모형을 가장 크게 개선한 것으로 생각되는 기법 유형은 무엇인가요?
3. 이 프로젝트를 통틀어 가장 어려움을 겪었던 문제에는 어떤 것들이 있나요?
4. 파이프라인에서 여러분이 이해할 수 없었던 부분 중 해결되지 않은 질문이 있나요?
5. 이 프로젝트를 진행하면서 기계 학습에 대해 배운 가장 중요한 세 가지는 무엇인가요?

### <span style="color: blue;">프로젝트 프레젠테이션: 이러한 질문에 대한 답변을 요약하여 프로젝트 프레젠테이션에도 추가합니다. 이제 프로젝트 프레젠테이션을 위한 모든 노트를 취합하여 학급을 대상으로 발표할 준비를 합니다.</span>